In [1]:
spark

In [2]:
dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2a.problematic_members_excluded/'
file_flag = 'msno_O-Z'

In [3]:
infile = dir0 + file_flag
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- tid: string (nullable = true)
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [4]:
df0.limit(5).toPandas()

,tid,msno,pay_method,plan_days,list_price,actual_paid,auto_renew,is_cancel,trans_date,expire_date,corr_exp_date,corr_start_date
0,25769803789,P/dXasg4zaZXoFfSZyw259eo1hA7wAtUwNdplkQ7EMU=,41,30,99,99,true,false,2016-08-15,2016-09-16,2016-09-16,2016-08-18
1,25769803797,ZLlb4auReNcABpweNJD+TAynZbZDd5iMBVaAUlOdBuk=,39,30,149,149,true,false,2016-03-06,2016-04-05,2016-04-05,2016-03-07
2,25769803801,Tqia4w8hGgZvH+M/x4gjXgVYqC47lxxPTX7xXQ+KgZk=,39,30,149,149,true,false,2015-12-02,2016-01-25,2016-01-25,2015-12-27
3,25769803803,YSYRycHE3MJit/trkXX20IIQQ6KFKMDpxFvDJPBvIDo=,41,30,99,99,true,false,2016-09-17,2016-10-17,2016-10-17,2016-09-18
4,25769803805,WozSFTAf2VkveP9KLCPm+eppBuU/2EGSZmjMGY+fb/8=,41,30,149,149,true,true,2016-04-19,2016-04-19,2016-04-19,None


In [5]:
from pyspark.sql.functions import col
df0.where(col('is_cancel') == 'true').show(1)

+-----------+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|        tid|                msno|pay_method|plan_days|list_price|actual_paid|auto_renew|is_cancel|trans_date|expire_date|corr_exp_date|corr_start_date|
+-----------+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
|25769804062|zmE7Pp4bDpEC0huG3...|        29|       30|       180|        180|      true|     true|2017-01-01| 2017-01-04|   2017-01-04|           null|
+-----------+--------------------+----------+---------+----------+-----------+----------+---------+----------+-----------+-------------+---------------+
only showing top 1 row



# Collect Infos for Each Member 

In [5]:
columns = ('tid', 'trans_date', 'is_cancel', 'corr_start_date', 'corr_exp_date')
df1 = df0.select('msno', *columns)
df1.show(5)

+--------------------+-----------+----------+---------+---------------+-------------+
|                msno|        tid|trans_date|is_cancel|corr_start_date|corr_exp_date|
+--------------------+-----------+----------+---------+---------------+-------------+
|P/dXasg4zaZXoFfSZ...|25769803789|2016-08-15|    false|     2016-08-18|   2016-09-16|
|ZLlb4auReNcABpweN...|25769803797|2016-03-06|    false|     2016-03-07|   2016-04-05|
|Tqia4w8hGgZvH+M/x...|25769803801|2015-12-02|    false|     2015-12-27|   2016-01-25|
|YSYRycHE3MJit/trk...|25769803803|2016-09-17|    false|     2016-09-18|   2016-10-17|
|WozSFTAf2VkveP9KL...|25769803805|2016-04-19|     true|           null|   2016-04-19|
+--------------------+-----------+----------+---------+---------------+-------------+
only showing top 5 rows



In [6]:
from pyspark.sql.functions import concat_ws
df2 = df1.fillna('na', ['corr_start_date']) \
      .select('msno',
              concat_ws('/', *columns).alias('info'))

df2.show(5, truncate=False)

+--------------------------------------------+--------------------------------------------------+
|msno                                        |info                                              |
+--------------------------------------------+--------------------------------------------------+
|P/dXasg4zaZXoFfSZyw259eo1hA7wAtUwNdplkQ7EMU=|25769803789/2016-08-15/false/2016-08-18/2016-09-16|
|ZLlb4auReNcABpweNJD+TAynZbZDd5iMBVaAUlOdBuk=|25769803797/2016-03-06/false/2016-03-07/2016-04-05|
|Tqia4w8hGgZvH+M/x4gjXgVYqC47lxxPTX7xXQ+KgZk=|25769803801/2015-12-02/false/2015-12-27/2016-01-25|
|YSYRycHE3MJit/trkXX20IIQQ6KFKMDpxFvDJPBvIDo=|25769803803/2016-09-17/false/2016-09-18/2016-10-17|
|WozSFTAf2VkveP9KLCPm+eppBuU/2EGSZmjMGY+fb/8=|25769803805/2016-04-19/true/na/2016-04-19         |
+--------------------------------------------+--------------------------------------------------+
only showing top 5 rows



In [7]:
from pyspark.sql.functions import concat_ws, collect_list
df3 = df2.groupBy('msno').agg(concat_ws('|', collect_list('info')).alias('infos'))

#df3.show(1)

In [ ]:
# output (temp)
outfile = dir0 + 'temp'
df3.write.format('csv').option('header','true').save(outfile)

# Reduce Transaction Records to Effective Subscriptions

In [3]:
# input (temp)
infile = dir0 + 'temp'
df0 = spark.read.format('csv').option('header','true').load(infile)
rdd0 = df0.rdd

In [4]:
rdd0.take(4)

[Row(msno='O/LxEiG/IAunjfLYvCm8kbgr0OcaAUX3g0pTAlzUMn0=', infos='94489799138/2016-12-27/false/2016-12-29/2017-01-27|34360112087/2015-12-27/false/2015-12-29/2016-01-27|94489899242/2015-11-27/false/2015-11-28/2015-12-27|179524/2017-02-27/false/2017-02-27/2017-03-27|60129780946/2016-10-27/false/2016-10-29/2016-11-27|621588/2016-11-27/false/2016-11-28/2016-12-27|60129953556/2016-08-27/false/2016-08-29/2016-09-27|103079471401/2016-02-27/false/2016-02-27/2016-03-27|103079502981/2017-01-27/false/2017-01-29/2017-02-27|103079784102/2016-05-27/false/2016-05-29/2016-06-27|111669647025/2016-09-27/false/2016-09-28/2016-10-27|146028923186/2016-01-27/false/2016-01-29/2016-02-27|68719688951/2016-07-27/false/2016-07-29/2016-08-27|25769898932/2016-04-27/false/2016-04-28/2016-05-27|25769995663/2016-06-27/false/2016-06-28/2016-07-27|42949833798/2016-03-27/false/2016-03-29/2016-04-27'),
 Row(msno='O/OD9sBB2fJ6r0cY7XcCALK6m534fWh9pTzJ+DZ/Cx4=', infos='34360224687/2016-09-20/false/2016-09-21/2016-10-20|94489

In [5]:
import datetime as DT

def add_one_day(date_str):
    t = DT.datetime.strptime(date_str, '%Y-%m-%d') + DT.timedelta(days=1)
    return t.strftime('%Y-%m-%d')


def subtract_one_day(date_str):
    t = DT.datetime.strptime(date_str, '%Y-%m-%d') - DT.timedelta(days=1)
    return t.strftime('%Y-%m-%d')


def find_indeterminate_period(tinfo1, tinfo2):
    tinfo_cancel, tinfo_noncancel = (tinfo1, tinfo2)  if tinfo1[2] == 'true'  else (tinfo2, tinfo1)
    dc = tinfo_cancel[4]
    if dc >= tinfo_noncancel[4]:
        return None
    elif dc >= tinfo_noncancel[3]:
        return (add_one_day(dc), tinfo_noncancel[4])
    else: # dc < tinfo_noncancel[3]
        return (tinfo_noncancel[3], tinfo_noncancel[4])

    
def reduce_subscription_periods(subscrip_periods, cut_date):
    '''
    cut all periods in `subscrip_periods` to before `cut_date`
    '''
    subscrip_periods_copy = subscrip_periods[:]
    pre_cut_date = subtract_one_day(cut_date)
    
    for p in subscrip_periods_copy:
        if cut_date <= p[1] :
            subscrip_periods.remove(p)
            
    for i in range(0, len(subscrip_periods)):
        p = subscrip_periods[i]
        if cut_date <= p[2]:
            subscrip_periods[i] = (p[0], p[1], pre_cut_date)
            

def has_no_cancel(tinfo_records):
    r = True
    for is_cancel in map(lambda info_rec: info_rec[2]=='true', tinfo_records):
        if is_cancel:
            r = False
            break
    return r


def reduce_trans_infos(trans_infos_str):
    '''
    Deals with cancels and determines effective subscription periods.
    Returns (subscription-periods, indeterminate-periods).
    '''
    trans_infos_strlist = trans_infos_str.split('|')
    trans_infos = [s.split('/')  for s in trans_infos_strlist]
        #
        # trans_infos[trans-index][item-index]  
        # item-index  --  0: tid;  1: transaction date;  2: is cancel;  
        #                 3: start date;  4: end (expiration) date
    
    ##
    if has_no_cancel(trans_infos):
        subscrip_periods = [(tinfo[0], tinfo[3], tinfo[4])  for tinfo in trans_infos]
        return (subscrip_periods, [])
            
    ## separate the transaction records by transaction dates having a cancel
    ### get transaction dates having a cancel
    trans_dates_w_cancel_s = set()
    last_trans_date = '1970-01-01'
    for tinfos in trans_infos:
        if tinfos[2] == 'true':
            trans_dates_w_cancel_s.add(tinfos[1])
        if tinfos[1] > last_trans_date:
            last_trans_date = tinfos[1]
        
    separating_dates = sorted(list(trans_dates_w_cancel_s)) + [last_trans_date]

    ###
    trans_info_groups = []
    for sdate in separating_dates:
        group = [tinfos  for tinfos in trans_infos  if tinfos[1] <= sdate]
        if group != []:
            trans_info_groups.append(group)
            for info in group:
                trans_infos.remove(info)
        # now `trans_infos` is destroyed
        # use `trans_info_groups` instead
        
    ## deal with each group
    subscrip_periods = []
    indeterm_periods = []
    for tinfos in trans_info_groups:
        assert tinfos != []
        
        ## sort by transaction date
        tinfos.sort(key=(lambda tinfo: tinfo[1]))
                
        ##
        if has_no_cancel(tinfos):
            subscrip_periods_1 = [(tinfo[0], tinfo[3], tinfo[4])  for tinfo in tinfos]
            subscrip_periods.extend(subscrip_periods_1)
            continue
            
        ## get transaction infos before last transaction date
        last_tdate = tinfos[-1][1]
        
        ind_pre_last_date = -1
        for i in range(len(tinfos)-1, -1, -1):
            if tinfos[i][1] != last_tdate:
                ind_pre_last_date = i
                break
        tinfos_pre_last_date = tinfos[0:ind_pre_last_date+1]  
        assert (ind_pre_last_date+1) < len(tinfos)
        
        ## transform `tinfos_pre_last_date` into subscription periods
        subscrip_periods_1 = [(tinfo[0], tinfo[3], tinfo[4])  for tinfo in tinfos_pre_last_date]
            
        ## find whether there is non-cancel transaction on the last transaction date
        has_non_cancel = any([tinfo[2]=='false'  for tinfo in tinfos[ind_pre_last_date+1:]])
        
        ##
        indet_period = None
        if has_non_cancel:  # one cancel and one non-cancel on `last_tdate`
            indet_period = find_indeterminate_period(tinfos[-2], tinfos[-1])
            if indet_period is not None:
                cut_date = indet_period[0]
                reduce_subscription_periods(subscrip_periods_1, cut_date)
        else:  # only cancels on `last_tdate`
            cut_date = min([tinfo[4] for tinfo in tinfos[ind_pre_last_date+1:]])
            reduce_subscription_periods(subscrip_periods_1, cut_date)
        
        ## store results
        subscrip_periods.extend(subscrip_periods_1)
        if indet_period is not None:
            indeterm_periods.append(indet_period)
    ## END of looping trans_info_groups
    
    return (subscrip_periods, indeterm_periods)


In [103]:
trans_infos = '51539718401/2016-05-05/false/2016-06-08/2016-07-07|51540051214/2016-07-05/false/2016-08-08/2016-09-06|' \
  + '51540124967/2016-09-05/false/2016-10-08/2016-11-06|51540369529/2016-01-24/false/2016-02-27/2016-03-27|' \
  + '8589999995/2016-11-05/false/2016-12-08/2017-01-06|94489594123/2016-02-06/true/na/2016-03-08|' \
  + '94490024042/2017-02-05/false/2017-02-05/2017-03-06|146029241083/2015-12-24/false/2016-01-27/2016-02-25|' \
  + '103079250926/2016-04-05/false/2016-05-08/2016-06-06|103079361450/2016-03-05/false/2016-04-08/2016-05-07|' \
  + '111669172613/2016-02-06/false/2016-03-08/2016-04-06|154619145884/2017-01-05/false/2017-01-08/2017-02-06|' \
  + '154619268354/2016-08-05/false/2016-09-08/2016-10-07|17180245295/2015-11-24/false/2015-12-27/2016-01-25|' \
  + '60129868029/2016-10-05/false/2016-11-08/2016-12-07|85899981601/2016-06-05/false/2016-07-08/2016-08-06'
reduce_trans_infos(trans_infos)

([('17180245295', '2015-12-27', '2016-01-25'),
  ('146029241083', '2016-01-27', '2016-02-25'),
  ('51540369529', '2016-02-27', '2016-03-08'),
  ('103079361450', '2016-04-08', '2016-05-07'),
  ('103079250926', '2016-05-08', '2016-06-06'),
  ('51539718401', '2016-06-08', '2016-07-07'),
  ('85899981601', '2016-07-08', '2016-08-06'),
  ('51540051214', '2016-08-08', '2016-09-06'),
  ('154619268354', '2016-09-08', '2016-10-07'),
  ('51540124967', '2016-10-08', '2016-11-06'),
  ('60129868029', '2016-11-08', '2016-12-07'),
  ('8589999995', '2016-12-08', '2017-01-06'),
  ('154619145884', '2017-01-08', '2017-02-06'),
  ('94490024042', '2017-02-05', '2017-03-06')],
 [('2016-03-09', '2016-04-06')])

In [6]:
def pack_data(msno, subscrip_periods, indeterm_periods):
    return [(msno, it[0], it[1], it[2])  for it in subscrip_periods] \
            + [(msno, 'indeterm', it[0], it[1])  for it in indeterm_periods]

#pack_data('--msno--', *reduce_trans_infos(trans_infos))

In [7]:
rdd1 = rdd0.flatMap(lambda row: pack_data(row['msno'], *reduce_trans_infos(row['infos'])))
rdd2 = rdd1.map(lambda rec: ','.join(rec))

rdd2.take(3)

['O/LxEiG/IAunjfLYvCm8kbgr0OcaAUX3g0pTAlzUMn0=,94489799138,2016-12-29,2017-01-27',
 'O/LxEiG/IAunjfLYvCm8kbgr0OcaAUX3g0pTAlzUMn0=,34360112087,2015-12-29,2016-01-27',
 'O/LxEiG/IAunjfLYvCm8kbgr0OcaAUX3g0pTAlzUMn0=,94489899242,2015-11-28,2015-12-27']

In [8]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/3.subscription_periods/' \
          + file_flag
rdd2.saveAsTextFile(outfile)

# gather files

In [7]:
dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/3.subscription_periods/'

df0 = spark.read.format('csv').option('header','false').load(dir0 + 'msno_a-n') \
      .selectExpr('_c0 AS msno', '_c1 AS tid', '_c2 AS start_date', '_c3 AS end_date')

In [8]:
df0.show(3)

+--------------------+-----------+----------+----------+
|                msno|        tid|start_date|  end_date|
+--------------------+-----------+----------+----------+
|a+EoQXBVpD4ucwTTm...|25769896722|2016-07-28|2016-08-26|
|a+EoQXBVpD4ucwTTm...|77309458613|2016-03-25|2016-04-23|
|a+EoQXBVpD4ucwTTm...|77309667985|2016-11-29|2016-12-28|
+--------------------+-----------+----------+----------+
only showing top 3 rows



In [9]:
df1 = spark.read.format('csv').option('header','false').load(dir0 + 'msno_A-N') \
      .selectExpr('_c0 AS msno', '_c1 AS tid', '_c2 AS start_date', '_c3 AS end_date')
df1.show(3)

+--------------------+------------+----------+----------+
|                msno|         tid|start_date|  end_date|
+--------------------+------------+----------+----------+
|A+1I7YjS1qQgXS5Si...|103079222190|2017-02-20|2017-03-21|
|A+1I7YjS1qQgXS5Si...|103079323436|2016-12-23|2017-01-21|
|A+1I7YjS1qQgXS5Si...|154619240845|2017-01-23|2017-02-21|
+--------------------+------------+----------+----------+
only showing top 3 rows



In [10]:
df2 = spark.read.format('csv').option('header','false').load(dir0 + 'msno_non-letter') \
      .selectExpr('_c0 AS msno', '_c1 AS tid', '_c2 AS start_date', '_c3 AS end_date')
df2.show(3)

+--------------------+-----------+----------+----------+
|                msno|        tid|start_date|  end_date|
+--------------------+-----------+----------+----------+
|++5nB0VuUuyj9xlng...|77309684417|2016-02-06|2016-03-03|
|++5nB0VuUuyj9xlng...|85899643848|2016-03-13|2016-04-11|
|++5nB0VuUuyj9xlng...|17180114820|2016-04-13|2016-05-12|
+--------------------+-----------+----------+----------+
only showing top 3 rows



In [11]:
df3 = spark.read.format('csv').option('header','false').load(dir0 + 'msno_o-z') \
      .selectExpr('_c0 AS msno', '_c1 AS tid', '_c2 AS start_date', '_c3 AS end_date')
df3.show(3)

+--------------------+------------+----------+----------+
|                msno|         tid|start_date|  end_date|
+--------------------+------------+----------+----------+
|o+73fwNA9WxP9ExWw...|146029075362|2016-06-01|2016-06-05|
|o+H0+uKG2ulOMTYWC...| 94489981585|2016-12-02|2016-12-31|
|o+H0+uKG2ulOMTYWC...| 51539820060|2017-03-02|2017-03-31|
+--------------------+------------+----------+----------+
only showing top 3 rows



In [12]:
df4 = spark.read.format('csv').option('header','false').load(dir0 + 'msno_O-Z') \
      .selectExpr('_c0 AS msno', '_c1 AS tid', '_c2 AS start_date', '_c3 AS end_date')
df4.show(3)

+--------------------+-----------+----------+----------+
|                msno|        tid|start_date|  end_date|
+--------------------+-----------+----------+----------+
|O/LxEiG/IAunjfLYv...|94489799138|2016-12-29|2017-01-27|
|O/LxEiG/IAunjfLYv...|34360112087|2015-12-29|2016-01-27|
|O/LxEiG/IAunjfLYv...|94489899242|2015-11-28|2015-12-27|
+--------------------+-----------+----------+----------+
only showing top 3 rows



In [13]:
df_all = df0.union(df1).union(df2).union(df3).union(df4)

df_all.show(10)

+--------------------+------------+----------+----------+
|                msno|         tid|start_date|  end_date|
+--------------------+------------+----------+----------+
|a+EoQXBVpD4ucwTTm...| 25769896722|2016-07-28|2016-08-26|
|a+EoQXBVpD4ucwTTm...| 77309458613|2016-03-25|2016-04-23|
|a+EoQXBVpD4ucwTTm...| 77309667985|2016-11-29|2016-12-28|
|a+EoQXBVpD4ucwTTm...| 77310025766|2016-12-30|2017-01-28|
|a+EoQXBVpD4ucwTTm...| 77310169372|2016-05-26|2016-06-24|
|a+EoQXBVpD4ucwTTm...|  8590579205|2016-04-25|2016-05-24|
|a+EoQXBVpD4ucwTTm...|103079882220|2016-06-26|2016-07-25|
|a+EoQXBVpD4ucwTTm...|120259373437|2016-09-29|2016-10-28|
|a+EoQXBVpD4ucwTTm...|111669505952|2016-02-24|2016-03-24|
|a+EoQXBVpD4ucwTTm...|128849500308|2016-02-12|2016-02-21|
+--------------------+------------+----------+----------+
only showing top 10 rows



In [14]:
# output
outfile = dir0 + 'all'
df_all.write.format('csv').option('header', 'true').save(outfile)